In [1]:
import numpy as np
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import (
    GeomAbs_Line, GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Hyperbola,
    GeomAbs_Parabola, GeomAbs_BezierCurve, GeomAbs_BSplineCurve,
    GeomAbs_OffsetCurve, GeomAbs_OtherCurve
)
from OCC.Extend.TopologyUtils import TopologyExplorer

# Import STEP file
step_file_path = r"c:\Users\serba\OneDrive - KU Leuven\master's fixturing topic\Parts for Validation\Part4.stp" # Change this to your STEP file path
step_reader = STEPControl_Reader()
status = step_reader.ReadFile(step_file_path)
if status != 1:
    raise IOError("Failed to read STEP file")
step_reader.TransferRoots()
shape = step_reader.OneShape()

# Initialize topology explorer
topo_explorer = TopologyExplorer(shape)

# Store results
edge_types = {}
edge_lengths = []

# Iterate through edges
for edge in topo_explorer.edges():
    
    # ===== FINDING EDGE TYPE =====
    curve_adaptor = BRepAdaptor_Curve(edge)
    curve_type = curve_adaptor.GetType()
    
    type_names = {
        GeomAbs_Line: 'Line',
        GeomAbs_Circle: 'Circle',
        GeomAbs_Ellipse: 'Ellipse',
        GeomAbs_Hyperbola: 'Hyperbola',
        GeomAbs_Parabola: 'Parabola',
        GeomAbs_BezierCurve: 'Bezier',
        GeomAbs_BSplineCurve: 'BSpline',
        GeomAbs_OffsetCurve: 'Offset',
        GeomAbs_OtherCurve: 'Other'
    }
    
    curve_type_name = type_names.get(curve_type, 'Unknown')
    is_straight = curve_type == GeomAbs_Line
    
    # ===== FINDING EDGE LENGTH =====
    first_param = curve_adaptor.FirstParameter()
    last_param = curve_adaptor.LastParameter()
    
    num_points = 50 if not is_straight else 2
    params = np.linspace(first_param, last_param, num_points)
    
    points = []
    for param in params:
        pnt = curve_adaptor.Value(param)
        points.append([pnt.X(), pnt.Y(), pnt.Z()])
    
    points = np.array(points)
    
    edge_length = 0
    for j in range(len(points) - 1):
        edge_length += np.linalg.norm(points[j+1] - points[j])
    
    # Store results
    edge_types[curve_type_name] = edge_types.get(curve_type_name, 0) + 1
    edge_lengths.append(edge_length)

# Print results
print(f"\n{'='*50}")
print(f"EDGE ANALYSIS RESULTS")
print(f"{'='*50}")
print(f"\nTotal Edges: {len(edge_lengths)}")
print(f"\nEdge Type Distribution:")
for edge_type, count in sorted(edge_types.items()):
    print(f"  {edge_type}: {count}")
print(f"\nEdge Length Statistics:")
print(f"  Min Length: {min(edge_lengths):.3f}")
print(f"  Max Length: {max(edge_lengths):.3f}")
print(f"  Average Length: {np.mean(edge_lengths):.3f}")
print(f"  Total Length: {sum(edge_lengths):.3f}")
print(f"\nExample Lengths (first 10 edges):")
for i, length in enumerate(edge_lengths[:10]):
    print(f"  Edge {i}: {length:.3f}")


EDGE ANALYSIS RESULTS

Total Edges: 141

Edge Type Distribution:
  BSpline: 2
  Circle: 4
  Line: 135

Edge Length Statistics:
  Min Length: 10.000
  Max Length: 156.972
  Average Length: 42.996
  Total Length: 6062.497

Example Lengths (first 10 edges):
  Edge 0: 20.000
  Edge 1: 30.000
  Edge 2: 10.000
  Edge 3: 100.000
  Edge 4: 30.000
  Edge 5: 70.000
  Edge 6: 10.000
  Edge 7: 30.000
  Edge 8: 10.000
  Edge 9: 30.000
